In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics import classification_report
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import joblib
import lazypredict

In [2]:
def reverse_ohe(row):
    if row['3'] == 1:
        return 3
    elif row['4'] == 1:
        return 4
    elif row['5'] == 1:
        return 5
    elif row['6'] == 1:
        return 6
    elif row['7'] == 1:
        return 7
    elif row['8'] == 1:
        return 8
# y_df['label'] = y_df.apply(reverse_ohe, axis=1)
# y_df

In [3]:
wine_red_x_train_file = "../datasets/winequality_red_x_train.csv"
wine_red_y_train_file = "../datasets/winequality_red_y_train.csv"

wine_red_x_train = pd.read_csv(wine_red_x_train_file)
wine_red_y_train = pd.read_csv(wine_red_y_train_file)

wine_red_y_train['label'] = wine_red_y_train.apply(reverse_ohe, axis=1)
wine_red_y_train

,3,4,5,6,7,8,label
0,0,0,0,1,0,0,6
1,0,0,0,1,0,0,6
2,0,0,1,0,0,0,5
3,0,0,0,0,1,0,7
4,0,0,1,0,0,0,5
...,...,...,...,...,...,...,...
1066,0,0,0,1,0,0,6
1067,0,0,0,1,0,0,6
1068,0,0,1,0,0,0,5
1069,0,0,0,0,1,0,7


In [6]:
x_train, x_test, y_train, y_test = train_test_split(wine_red_x_train, wine_red_y_train['label'], random_state=0, test_size=0.2)

print(x_train.shape)
print(x_test.shape)

(856, 11)
(215, 11)


In [7]:
from sklearn.preprocessing import MinMaxScaler
import pickle

scaler = MinMaxScaler()
x_train_scld = pd.DataFrame(scaler.fit_transform(x_train), columns=x_train.columns)
x_test_scld = pd.DataFrame(scaler.transform(x_test), columns=x_test.columns)

x_train_scld = np.array(x_train_scld)
x_test_scld = np.array(x_test_scld)
y_train = np.array(y_train)
y_test = np.array(y_test)

# y_train

In [27]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier

In [28]:
classifiers = {
    "KNeighbors": KNeighborsClassifier(),
    "SVC": SVC(),
    "RandomForest": RandomForestClassifier(),
    "AdaBoost": AdaBoostClassifier(),
    "ExtraTreesClassifier": ExtraTreesClassifier()
}

model_evals_scaled = {
    "model": [],
    "train_score": [],
    "test_score": []
}

In [29]:
def plot_conf_matrix(conf_mtrx):
    fig, ax = plt.subplots(figsize=(5,5))
    ax.matshow(conf_mtrx, cmap=plt.cm.Oranges, alpha=0.3)
    
    for i in range(conf_mtrx.shape[0]):
        for j in range(conf_mtrx.shape[1]):
            ax.text(x=j, y=i, s=conf_mtrx[i,j], va='center', ha='center', size='xx-large')
    
    plt.xlabel("Predictions", fontsize=18)
    plt.ylabel("Actuals", fontsize=18)
    plt.title("Confusion Matrix", fontsize=18)
    
    plt.show()

In [30]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

for key, classifier in classifiers.items():
    print(key)
    classifier.fit(x_train, y_train)
    preds = classifier.predict(x_test)
    train_score = classifier.score(x_train, y_train)
#     rmse_ = rmse(preds, y_test)
#     precision = precision_score(y_test, preds)
#     recall = recall_score(y_test, preds)
#     f1 = f1_score(y_test, preds)
    test_acc = accuracy_score(y_test, preds)
    
    model_evals_scaled['model'].append(str(key))
    model_evals_scaled['train_score'].append(train_score.round(4))
    model_evals_scaled['test_score'].append(test_acc.round(4))
#     model_evals['rmse'].append(rmse_.round(4))
#     model_evals_balanced['precision'].append(precision.round(4))
#     model_evals_balanced['recall'].append(recall.round(4))
#     model_evals_balanced['f1_score'].append(f1.round(4))
    
    
#     conf_matrix = multilabel_confusion_matrix(y_true=y_test, y_pred=preds)
#     plot_conf_matrix(conf_matrix)
    
    print()
    print('-'*60)

model_vals = pd.DataFrame(model_evals_scaled)

KNeighbors

------------------------------------------------------------
SVC

------------------------------------------------------------
RandomForest

------------------------------------------------------------
AdaBoost

------------------------------------------------------------
ExtraTreesClassifier

------------------------------------------------------------


In [31]:
model_vals

,model,train_score,test_score
0,KNeighbors,0.69,0.55
1,SVC,0.65,0.63
2,RandomForest,1.00,0.68
3,AdaBoost,0.58,0.54
4,ExtraTreesClassifier,1.00,0.68


In [13]:
from sklearn.model_selection import GridSearchCV
  
param_grid = {'C': [0.1, 1, 10, 100, 1000], 
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf', 'poly', 'sigmoid']}
  
grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3)
  
grid.fit(x_train, y_train)

Fitting 5 folds for each of 75 candidates, totalling 375 fits
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.541 total time=   0.0s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.550 total time=   0.0s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.637 total time=   0.0s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.585 total time=   0.0s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.585 total time=   0.0s
[CV 1/5] END .......C=0.1, gamma=1, kernel=poly;, score=0.552 total time=   0.0s
[CV 2/5] END .......C=0.1, gamma=1, kernel=poly;, score=0.509 total time=   0.0s
[CV 3/5] END .......C=0.1, gamma=1, kernel=poly;, score=0.626 total time=   0.0s
[CV 4/5] END .......C=0.1, gamma=1, kernel=poly;, score=0.579 total time=   0.0s
[CV 5/5] END .......C=0.1, gamma=1, kernel=poly;, score=0.561 total time=   0.0s
[CV 1/5] END ....C=0.1, gamma=1, kernel=sigmoid;, score=0.453 total time=   0.0s
[CV 2/5] END ....C=0.1, gamma=1, kernel=sigmoid

[CV 4/5] END ....C=1, gamma=0.1, kernel=sigmoid;, score=0.585 total time=   0.0s
[CV 5/5] END ....C=1, gamma=0.1, kernel=sigmoid;, score=0.567 total time=   0.0s
[CV 1/5] END .......C=1, gamma=0.01, kernel=rbf;, score=0.419 total time=   0.0s
[CV 2/5] END .......C=1, gamma=0.01, kernel=rbf;, score=0.421 total time=   0.0s
[CV 3/5] END .......C=1, gamma=0.01, kernel=rbf;, score=0.421 total time=   0.0s
[CV 4/5] END .......C=1, gamma=0.01, kernel=rbf;, score=0.421 total time=   0.0s
[CV 5/5] END .......C=1, gamma=0.01, kernel=rbf;, score=0.421 total time=   0.0s
[CV 1/5] END ......C=1, gamma=0.01, kernel=poly;, score=0.419 total time=   0.0s
[CV 2/5] END ......C=1, gamma=0.01, kernel=poly;, score=0.421 total time=   0.0s
[CV 3/5] END ......C=1, gamma=0.01, kernel=poly;, score=0.421 total time=   0.0s
[CV 4/5] END ......C=1, gamma=0.01, kernel=poly;, score=0.421 total time=   0.0s
[CV 5/5] END ......C=1, gamma=0.01, kernel=poly;, score=0.421 total time=   0.0s
[CV 1/5] END ...C=1, gamma=0

[CV 2/5] END .C=10, gamma=0.001, kernel=sigmoid;, score=0.421 total time=   0.0s
[CV 3/5] END .C=10, gamma=0.001, kernel=sigmoid;, score=0.421 total time=   0.0s
[CV 4/5] END .C=10, gamma=0.001, kernel=sigmoid;, score=0.421 total time=   0.0s
[CV 5/5] END .C=10, gamma=0.001, kernel=sigmoid;, score=0.421 total time=   0.0s
[CV 1/5] END ....C=10, gamma=0.0001, kernel=rbf;, score=0.419 total time=   0.0s
[CV 2/5] END ....C=10, gamma=0.0001, kernel=rbf;, score=0.421 total time=   0.0s
[CV 3/5] END ....C=10, gamma=0.0001, kernel=rbf;, score=0.421 total time=   0.0s
[CV 4/5] END ....C=10, gamma=0.0001, kernel=rbf;, score=0.421 total time=   0.0s
[CV 5/5] END ....C=10, gamma=0.0001, kernel=rbf;, score=0.421 total time=   0.0s
[CV 1/5] END ...C=10, gamma=0.0001, kernel=poly;, score=0.419 total time=   0.0s
[CV 2/5] END ...C=10, gamma=0.0001, kernel=poly;, score=0.421 total time=   0.0s
[CV 3/5] END ...C=10, gamma=0.0001, kernel=poly;, score=0.421 total time=   0.0s
[CV 4/5] END ...C=10, gamma=

[CV 4/5] END ......C=1000, gamma=1, kernel=poly;, score=0.532 total time=   0.3s
[CV 5/5] END ......C=1000, gamma=1, kernel=poly;, score=0.596 total time=   0.4s
[CV 1/5] END ...C=1000, gamma=1, kernel=sigmoid;, score=0.483 total time=   0.0s
[CV 2/5] END ...C=1000, gamma=1, kernel=sigmoid;, score=0.474 total time=   0.0s
[CV 3/5] END ...C=1000, gamma=1, kernel=sigmoid;, score=0.474 total time=   0.0s
[CV 4/5] END ...C=1000, gamma=1, kernel=sigmoid;, score=0.462 total time=   0.0s
[CV 5/5] END ...C=1000, gamma=1, kernel=sigmoid;, score=0.491 total time=   0.0s
[CV 1/5] END .....C=1000, gamma=0.1, kernel=rbf;, score=0.581 total time=   0.0s
[CV 2/5] END .....C=1000, gamma=0.1, kernel=rbf;, score=0.532 total time=   0.0s
[CV 3/5] END .....C=1000, gamma=0.1, kernel=rbf;, score=0.637 total time=   0.0s
[CV 4/5] END .....C=1000, gamma=0.1, kernel=rbf;, score=0.620 total time=   0.0s
[CV 5/5] END .....C=1000, gamma=0.1, kernel=rbf;, score=0.579 total time=   0.0s
[CV 1/5] END ....C=1000, gam

GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf', 'poly', 'sigmoid']},
             verbose=3)

In [14]:

print(grid.best_params_)
  
print(grid.best_estimator_)

{'C': 10, 'gamma': 1, 'kernel': 'poly'}
SVC(C=10, gamma=1, kernel='poly')


In [15]:
clf = grid.best_estimator_
grid_predictions = clf.predict(x_test)
  
print(classification_report(y_test, grid_predictions))
print(accuracy_score(y_test, grid_predictions))

              precision    recall  f1-score   support

           3       0.00      0.00      0.00         1
           4       0.00      0.00      0.00         6
           5       0.69      0.74      0.72       104
           6       0.53      0.66      0.59        76
           7       1.00      0.32      0.49        28
           8       0.00      0.00      0.00         0

    accuracy                           0.63       215
   macro avg       0.37      0.29      0.30       215
weighted avg       0.65      0.63      0.62       215

0.6325581395348837


C:\Users\JBOTOY\vscode_projects\data_science\python_ml_projects\training\mltool\env\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\JBOTOY\vscode_projects\data_science\python_ml_projects\training\mltool\env\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\JBOTOY\vscode_projects\data_science\python_ml_projects\training\mltool\env\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted sample

In [16]:
model_file = "svc.joblib"
joblib.dump(clf, model_file)

['svc.joblib']

In [18]:
from lazypredict.Supervised import LazyClassifier

clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)
models, predictions = clf.fit(x_train_scld, x_test_scld, y_train, y_test)
models

100%|███████████████████████████████████████| 29/29 [00:01<00:00, 15.49it/s]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
DecisionTreeClassifier,0.59,0.42,None,0.59,0.01
LabelSpreading,0.62,0.40,None,0.62,0.04
LabelPropagation,0.62,0.40,None,0.62,0.03
ExtraTreesClassifier,0.69,0.39,None,0.67,0.17
RandomForestClassifier,0.68,0.39,None,0.67,0.27
ExtraTreeClassifier,0.60,0.39,None,0.61,0.01
LinearDiscriminantAnalysis,0.64,0.36,None,0.63,0.02
LGBMClassifier,0.62,0.36,None,0.61,0.33
BaggingClassifier,0.61,0.35,None,0.60,0.07


In [ ]:
from sklearn.model_selection import GridSearchCV
  
param_grid = {'n_estimators': [10, 20, 50, 100, 200, 500, 1000, 1200, 1500, 1800, 1900, 2000, 2100, 3000], 
              'criterion': ['gini', 'entropy'],
              'max_depth': [1, 2, 5, 8, 13, 21, 34, 53, 54, 55, 89, None],
              'min_samples_split': [2, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377],
              'min_samples_leaf': [1, 2, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377],
              'min_weight_fraction_leaf': [x / 10 for x in range(0, 6)],
              'max_features': ['auto', 'sqrt', 'log2', 2, 5, 8, 13, 21, 34, None],
              
             }
  
grid = GridSearchCV(ExtraTreesClassifier(), param_grid, refit = True, verbose = 3)
  
grid.fit(x_train, y_train)